In [1]:

import sqlite3
conn = sqlite3.connect(':memory:')

cur = conn.cursor()

cur.execute("""CREATE TABLE IF NOT EXISTS users(
   userid INT PRIMARY KEY,
   fname TEXT,
   lname TEXT,
   gender TEXT);
""")
conn.commit()


cur.execute("""INSERT INTO users(userid, fname, lname, gender) 
   VALUES('00001', 'Nik', 'Piepenbreier', 'male');""")

conn.commit()

cur.execute("SELECT * FROM users;")
one_result = cur.fetchone()
print(one_result)



(1, 'Nik', 'Piepenbreier', 'male')


---

In [2]:

import sqlite3
conn = sqlite3.connect('users.db')

In [3]:


cur = conn.cursor()

cur.execute("""CREATE TABLE IF NOT EXISTS users(
   userid INT PRIMARY KEY,
   fname TEXT,
   lname TEXT,
   gender TEXT);
""")
conn.commit()


cur.execute("""INSERT INTO users(userid, fname, lname, gender) 
   VALUES('00001', 'Nik', 'Piepenbreier', 'male');""")

conn.commit()

cur.execute("SELECT * FROM users;")
one_result = cur.fetchone()
print(one_result)

(1, 'Nik', 'Piepenbreier', 'male')


In [4]:
conn.close()

---

## sqlite-vec

In [5]:
!pip install sqlite-vec

In [ ]:

import sqlite3
from typing import Tuple, Optional, List, Any, Union, Iterable

import sqlite_vec 
from sqlite_vec import serialize_float32



conn = sqlite3.connect('sqlite_vec_demo.db')
conn.enable_load_extension(True)
sqlite_vec.load(conn)
conn.enable_load_extension(False)

In [7]:
conn.execute("SELECT sqlite_version(), vec_version()").fetchone()

('3.49.1', 'v0.1.6')

In [8]:
## create table:


items = [
    (1, [0.1, 0.1, 0.1, 0.1]),
    (2, [0.2, 0.2, 0.2, 0.2]),
    (3, [0.3, 0.3, 0.3, 0.3]),
    (4, [0.4, 0.4, 0.4, 0.4]),
    (5, [0.5, 0.5, 0.5, 0.5]),
]


conn.execute(""" 
CREATE VIRTUAL TABLE IF NOT EXISTS vec_data USING vec0(embedding float[4])

""")

conn.commit()



In [18]:
serialized_items =  [(item[0], serialize_float32(item[1])) for item in items]


serialized_items
                     

[(1, b'\xcd\xcc\xcc=\xcd\xcc\xcc=\xcd\xcc\xcc=\xcd\xcc\xcc='),
 (2, b'\xcd\xccL>\xcd\xccL>\xcd\xccL>\xcd\xccL>'),
 (3, b'\x9a\x99\x99>\x9a\x99\x99>\x9a\x99\x99>\x9a\x99\x99>'),
 (4, b'\xcd\xcc\xcc>\xcd\xcc\xcc>\xcd\xcc\xcc>\xcd\xcc\xcc>'),
 (5, b'\x00\x00\x00?\x00\x00\x00?\x00\x00\x00?\x00\x00\x00?')]

In [ ]:
# c = b'123'
# type(c)

bytes

In [21]:
for item_id, serialized_vector in serialized_items:
    conn.execute(
        f"""
            INSERT INTO vec_data (rowid, embedding)
            VALUES(?, ?)
        """,
        [item_id, serialized_vector]
    )



In [22]:
conn.commit()

In [ ]:
query_vector = [0.3, 0.3, 0.3, 0.3]
results = conn.execute(
    """
    SELECT 
        rowid, distance
    FROM vec_data
    WHERE embedding MATCH ?
    ORDER BY distance
    LIMIT ?
    """,
    [serialize_float32(query_vector), 5]

).fetchall()

for res in results:
    print(res)

(3, 0.0)
(4, 0.19999998807907104)
(2, 0.20000001788139343)
(5, 0.3999999761581421)
(1, 0.40000003576278687)


In [ ]:
# a = 10
# b = 5
# c = 4

# rows in cross join of a,b,c

In [33]:
query_vector = [0.3, 0.3, 0.3, 0.3]
results = conn.execute(
    """
    SELECT 
        a.rowid as vec1_id,
        b.rowid as vec2_id,
        vec_distance_l1(a.embedding, b.embedding) as l2_distance,
        vec_distance_l2(a.embedding, b.embedding) as l2_distance,
        vec_distance_cosine(a.embedding, b.embedding) as cosine_distance

    FROM vec_data a
    CROSS JOIN vec_data b
    WHERE a.rowid < b.rowid
    ORDER BY a.rowid, b.rowid
    LIMIT 5
    """

).fetchall()

for res in results:
    print(res)


(1, 2, 0.4000000059604645, 0.20000000298023224, 0.0)
(1, 3, 0.8000000417232513, 0.40000003576278687, 1.552204231813903e-08)
(1, 4, 1.2000000178813934, 0.6000000238418579, 0.0)
(1, 5, 1.5999999940395355, 0.800000011920929, 2.0489094865183688e-08)
(2, 3, 0.40000003576278687, 0.20000001788139343, 1.552204231813903e-08)


In [ ]:
# blogs -> readers page-> suggested articles-> are semantically similar to current article